# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [5]:
encoded[:100]

array([81, 52, 69, 37, 70,  1, 33, 75, 21, 67, 67, 67, 29, 69, 37, 37,  7,
       75, 15, 69, 11, 56, 61, 56,  1, 64, 75, 69, 33,  1, 75, 69, 61, 61,
       75, 69, 61, 56, 78,  1,  5, 75,  1, 53,  1, 33,  7, 75, 80, 60, 52,
       69, 37, 37,  7, 75, 15, 69, 11, 56, 61,  7, 75, 56, 64, 75, 80, 60,
       52, 69, 37, 37,  7, 75, 56, 60, 75, 56, 70, 64, 75, 13, 32, 60, 67,
       32, 69,  7, 62, 67, 67, 79, 53,  1, 33,  7, 70, 52, 56, 60])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [13]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [14]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [19]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
    batch_total_size = batch_size * seq_length
    n_batches = len(arr) // batch_total_size
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[0:(n_batches * batch_total_size)]
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:(n + seq_length)]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1] = x[:, 1:]
        col = n + seq_length
        if (col >= arr.shape[1]):
            col = 0
        y[:, -1] = arr[:, col]
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [20]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [21]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[81 52 69 37 70  1 33 75 21 67]
 [64 13 60 75 70 52 69 70 75 69]
 [ 1 60 49 75 13 33 75 69 75 15]
 [64 75 70 52  1 75 51 52 56  1]
 [75 64 69 32 75 52  1 33 75 70]
 [51 80 64 64 56 13 60 75 69 60]
 [75 25 60 60 69 75 52 69 49 75]
 [ 3 76 61 13 60 64 78  7 62 75]]

y
 [[52 69 37 70  1 33 75 21 67 67]
 [13 60 75 70 52 69 70 75 69 70]
 [60 49 75 13 33 75 69 75 15 13]
 [75 70 52  1 75 51 52 56  1 15]
 [64 69 32 75 52  1 33 75 70  1]
 [80 64 64 56 13 60 75 69 60 49]
 [25 60 60 69 75 52 69 49 75 64]
 [76 61 13 60 64 78  7 62 75 43]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [24]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [25]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        n_chars = len(self.chars)
        self.lstm = nn.LSTM(n_chars, n_hidden, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(p=drop_prob)
        self.fc = nn.Linear(n_hidden, n_chars)
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        lstm_out, hidden = self.lstm(x, hidden)
        
        out = self.dropout(lstm_out)
        
        out = out.contiguous().view(-1, self.n_hidden)
        
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [29]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                # net.eval() will set all the layers in your model to evaluation mode. 
                # This affects layers like dropout layers that turn "off" nodes during training 
                # with some probability, but should allow every node to be "on" for evaluation.  
                # Set net.train()` (training mode) only during the training loop.
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [30]:
## TODO: set your model hyperparameters
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [31]:
batch_size = 128
seq_length = 100
n_epochs = 30def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
    batch_total_size = batch_size * seq_length
    n_batches = len(arr) // batch_total_size
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[0:(n_batches * batch_total_size)]
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:(n + seq_length)]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1] = x[:, 1:]
        col = n + seq_length
        if (col >= arr.shape[1]):
            col = 0
        y[:, -1] = arr[:, col]
        yield x, y # start small if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/2... Step: 10... Loss: 3.2577... Val Loss: 3.1715
Epoch: 1/2... Step: 20... Loss: 3.1481... Val Loss: 3.1323
Epoch: 1/2... Step: 30... Loss: 3.1442... Val Loss: 3.1199
Epoch: 1/2... Step: 40... Loss: 3.1128... Val Loss: 3.1189
Epoch: 1/2... Step: 50... Loss: 3.1377... Val Loss: 3.1165
Epoch: 1/2... Step: 60... Loss: 3.1147... Val Loss: 3.1123
Epoch: 1/2... Step: 70... Loss: 3.1006... Val Loss: 3.1058
Epoch: 1/2... Step: 80... Loss: 3.1033... Val Loss: 3.0892
Epoch: 1/2... Step: 90... Loss: 3.0751... Val Loss: 3.0485
Epoch: 1/2... Step: 100... Loss: 3.0033... Val Loss: 2.9620
Epoch: 1/2... Step: 110... Loss: 2.8808... Val Loss: 2.9093
Epoch: 1/2... Step: 120... Loss: 2.7726... Val Loss: 2.7675
Epoch: 1/2... Step: 130... Loss: 2.7010... Val Loss: 2.6611
Epoch: 2/2... Step: 140... Loss: 2.6269... Val Loss: 2.5629
Epoch: 2/2... Step: 150... Loss: 2.5649... Val Loss: 2.5102
Epoch: 2/2... Step: 160... Loss: 2.5043... Val Loss: 2.4673
Epoch: 2/2... Step: 170... Loss: 2.4420... Val Lo

Epoch: 1/30... Step: 10... Loss: 3.2600... Val Loss: 3.1890
Epoch: 1/30... Step: 20... Loss: 3.1447... Val Loss: 3.1314
Epoch: 1/30... Step: 30... Loss: 3.1374... Val Loss: 3.1201
Epoch: 1/30... Step: 40... Loss: 3.1119... Val Loss: 3.1191
Epoch: 1/30... Step: 50... Loss: 3.1414... Val Loss: 3.1162
Epoch: 1/30... Step: 60... Loss: 3.1178... Val Loss: 3.1130
Epoch: 1/30... Step: 70... Loss: 3.1002... Val Loss: 3.1072
Epoch: 1/30... Step: 80... Loss: 3.1054... Val Loss: 3.0917
Epoch: 1/30... Step: 90... Loss: 3.0845... Val Loss: 3.0619
Epoch: 1/30... Step: 100... Loss: 3.0086... Val Loss: 2.9822
Epoch: 1/30... Step: 110... Loss: 2.9154... Val Loss: 2.8893
Epoch: 1/30... Step: 120... Loss: 2.8092... Val Loss: 2.7952
Epoch: 1/30... Step: 130... Loss: 2.7381... Val Loss: 2.6800
Epoch: 2/30... Step: 140... Loss: 2.6353... Val Loss: 2.5939
Epoch: 2/30... Step: 150... Loss: 2.5816... Val Loss: 2.5194
Epoch: 2/30... Step: 160... Loss: 2.5176... Val Loss: 2.4772
Epoch: 2/30... Step: 170... Loss: 2.4568... Val Loss: 2.4438
Epoch: 2/30... Step: 180... Loss: 2.4330... Val Loss: 2.4069
Epoch: 2/30... Step: 190... Loss: 2.3837... Val Loss: 2.3763
Epoch: 2/30... Step: 200... Loss: 2.3736... Val Loss: 2.3429
Epoch: 2/30... Step: 210... Loss: 2.3434... Val Loss: 2.3211
Epoch: 2/30... Step: 220... Loss: 2.3079... Val Loss: 2.2861
Epoch: 2/30... Step: 230... Loss: 2.2911... Val Loss: 2.2665
Epoch: 2/30... Step: 240... Loss: 2.2636... Val Loss: 2.2347
Epoch: 2/30... Step: 250... Loss: 2.2114... Val Loss: 2.2030
Epoch: 2/30... Step: 260... Loss: 2.1790... Val Loss: 2.1799
Epoch: 2/30... Step: 270... Loss: 2.1912... Val Loss: 2.1582
Epoch: 3/30... Step: 280... Loss: 2.1818... Val Loss: 2.1368
Epoch: 3/30... Step: 290... Loss: 2.1563... Val Loss: 2.1116
Epoch: 3/30... Step: 300... Loss: 2.1204... Val Loss: 2.0924
Epoch: 3/30... Step: 310... Loss: 2.0976... Val Loss: 2.0712
Epoch: 3/30... Step: 320... Loss: 2.0672... Val Loss: 2.0478
Epoch: 3/30... Step: 330... Loss: 2.0288... Val Loss: 2.0344
Epoch: 3/30... Step: 340... Loss: 2.0618... Val Loss: 2.0187
Epoch: 3/30... Step: 350... Loss: 2.0348... Val Loss: 1.9997
Epoch: 3/30... Step: 360... Loss: 1.9728... Val Loss: 1.9812
Epoch: 3/30... Step: 370... Loss: 2.0020... Val Loss: 1.9692
Epoch: 3/30... Step: 380... Loss: 1.9717... Val Loss: 1.9515
Epoch: 3/30... Step: 390... Loss: 1.9376... Val Loss: 1.9364
Epoch: 3/30... Step: 400... Loss: 1.9239... Val Loss: 1.9230
Epoch: 3/30... Step: 410... Loss: 1.9352... Val Loss: 1.9050
Epoch: 4/30... Step: 420... Loss: 1.9238... Val Loss: 1.8943
Epoch: 4/30... Step: 430... Loss: 1.9095... Val Loss: 1.8780
Epoch: 4/30... Step: 440... Loss: 1.8947... Val Loss: 1.8665
Epoch: 4/30... Step: 450... Loss: 1.8419... Val Loss: 1.8502
Epoch: 4/30... Step: 460... Loss: 1.8281... Val Loss: 1.8439
Epoch: 4/30... Step: 470... Loss: 1.8673... Val Loss: 1.8365
Epoch: 4/30... Step: 480... Loss: 1.8413... Val Loss: 1.8198
Epoch: 4/30... Step: 490... Loss: 1.8416... Val Loss: 1.8086
Epoch: 4/30... Step: 500... Loss: 1.8483... Val Loss: 1.7973
Epoch: 4/30... Step: 510... Loss: 1.8040... Val Loss: 1.7887
Epoch: 4/30... Step: 520... Loss: 1.8218... Val Loss: 1.7771
Epoch: 4/30... Step: 530... Loss: 1.7796... Val Loss: 1.7667
Epoch: 4/30... Step: 540... Loss: 1.7481... Val Loss: 1.7553
Epoch: 4/30... Step: 550... Loss: 1.7940... Val Loss: 1.7458
Epoch: 5/30... Step: 560... Loss: 1.7530... Val Loss: 1.7381
Epoch: 5/30... Step: 570... Loss: 1.7384... Val Loss: 1.7310
Epoch: 5/30... Step: 580... Loss: 1.7273... Val Loss: 1.7241
Epoch: 5/30... Step: 590... Loss: 1.7230... Val Loss: 1.7121
Epoch: 5/30... Step: 600... Loss: 1.7229... Val Loss: 1.7092
Epoch: 5/30... Step: 610... Loss: 1.7082... Val Loss: 1.6982
Epoch: 5/30... Step: 620... Loss: 1.7102... Val Loss: 1.6935
Epoch: 5/30... Step: 630... Loss: 1.7145... Val Loss: 1.6865
Epoch: 5/30... Step: 640... Loss: 1.6834... Val Loss: 1.6778
Epoch: 5/30... Step: 650... Loss: 1.6787... Val Loss: 1.6710
Epoch: 5/30... Step: 660... Loss: 1.6540... Val Loss: 1.6662
Epoch: 5/30... Step: 670... Loss: 1.6831... Val Loss: 1.6608
Epoch: 5/30... Step: 680... Loss: 1.6739... Val Loss: 1.6504
Epoch: 5/30... Step: 690... Loss: 1.6463... Val Loss: 1.6422
Epoch: 6/30... Step: 700... Loss: 1.6488... Val Loss: 1.6373
Epoch: 6/30... Step: 710... Loss: 1.6308... Val Loss: 1.6319
Epoch: 6/30... Step: 720... Loss: 1.6271... Val Loss: 1.6273
Epoch: 6/30... Step: 730... Loss: 1.6488... Val Loss: 1.6197
Epoch: 6/30... Step: 740... Loss: 1.6051... Val Loss: 1.6176
Epoch: 6/30... Step: 750... Loss: 1.5953... Val Loss: 1.6089
Epoch: 6/30... Step: 760... Loss: 1.6328... Val Loss: 1.6066
Epoch: 6/30... Step: 770... Loss: 1.6175... Val Loss: 1.6007
Epoch: 6/30... Step: 780... Loss: 1.5983... Val Loss: 1.5942
Epoch: 6/30... Step: 790... Loss: 1.5900... Val Loss: 1.5895
Epoch: 6/30... Step: 800... Loss: 1.5999... Val Loss: 1.5858
Epoch: 6/30... Step: 810... Loss: 1.5836... Val Loss: 1.5783
Epoch: 6/30... Step: 820... Loss: 1.5410... Val Loss: 1.5739
Epoch: 6/30... Step: 830... Loss: 1.5925... Val Loss: 1.5703
Epoch: 7/30... Step: 840... Loss: 1.5442... Val Loss: 1.5650
Epoch: 7/30... Step: 850... Loss: 1.5626... Val Loss: 1.5595
Epoch: 7/30... Step: 860... Loss: 1.5479... Val Loss: 1.5544
Epoch: 7/30... Step: 870... Loss: 1.5559... Val Loss: 1.5494
Epoch: 7/30... Step: 880... Loss: 1.5703... Val Loss: 1.5483
Epoch: 7/30... Step: 890... Loss: 1.5535... Val Loss: 1.5440
Epoch: 7/30... Step: 900... Loss: 1.5375... Val Loss: 1.5421
Epoch: 7/30... Step: 910... Loss: 1.5104... Val Loss: 1.5388
Epoch: 7/30... Step: 920... Loss: 1.5359... Val Loss: 1.5304
Epoch: 7/30... Step: 930... Loss: 1.5230... Val Loss: 1.5280
Epoch: 7/30... Step: 940... Loss: 1.5233... Val Loss: 1.5227
Epoch: 7/30... Step: 950... Loss: 1.5352... Val Loss: 1.5194
Epoch: 7/30... Step: 960... Loss: 1.5353... Val Loss: 1.5150
Epoch: 7/30... Step: 970... Loss: 1.5354... Val Loss: 1.5135
Epoch: 8/30... Step: 980... Loss: 1.5008... Val Loss: 1.5079
Epoch: 8/30... Step: 990... Loss: 1.5038... Val Loss: 1.5056
Epoch: 8/30... Step: 1000... Loss: 1.4971... Val Loss: 1.5039
Epoch: 8/30... Step: 1010... Loss: 1.5328... Val Loss: 1.4972
Epoch: 8/30... Step: 1020... Loss: 1.4994... Val Loss: 1.4961
Epoch: 8/30... Step: 1030... Loss: 1.4815... Val Loss: 1.4910
Epoch: 8/30... Step: 1040... Loss: 1.4976... Val Loss: 1.4896
Epoch: 8/30... Step: 1050... Loss: 1.4820... Val Loss: 1.4932
Epoch: 8/30... Step: 1060... Loss: 1.4804... Val Loss: 1.4843
Epoch: 8/30... Step: 1070... Loss: 1.4881... Val Loss: 1.4829
Epoch: 8/30... Step: 1080... Loss: 1.4793... Val Loss: 1.4799
Epoch: 8/30... Step: 1090... Loss: 1.4599... Val Loss: 1.4747
Epoch: 8/30... Step: 1100... Loss: 1.4562... Val Loss: 1.4695
Epoch: 8/30... Step: 1110... Loss: 1.4681... Val Loss: 1.4695
Epoch: 9/30... Step: 1120... Loss: 1.4755... Val Loss: 1.4677
Epoch: 9/30... Step: 1130... Loss: 1.4705... Val Loss: 1.4661
Epoch: 9/30... Step: 1140... Loss: 1.4710... Val Loss: 1.4604
Epoch: 9/30... Step: 1150... Loss: 1.4808... Val Loss: 1.4610
Epoch: 9/30... Step: 1160... Loss: 1.4401... Val Loss: 1.4588
Epoch: 9/30... Step: 1170... Loss: 1.4520... Val Loss: 1.4549
Epoch: 9/30... Step: 1180... Loss: 1.4358... Val Loss: 1.4562
Epoch: 9/30... Step: 1190... Loss: 1.4740... Val Loss: 1.4525
Epoch: 9/30... Step: 1200... Loss: 1.4225... Val Loss: 1.4469
Epoch: 9/30... Step: 1210... Loss: 1.4414... Val Loss: 1.4474
Epoch: 9/30... Step: 1220... Loss: 1.4318... Val Loss: 1.4436
Epoch: 9/30... Step: 1230... Loss: 1.4036... Val Loss: 1.4417
Epoch: 9/30... Step: 1240... Loss: 1.4252... Val Loss: 1.4339
Epoch: 9/30... Step: 1250... Loss: 1.4343... Val Loss: 1.4372
Epoch: 10/30... Step: 1260... Loss: 1.4305... Val Loss: 1.4382
Epoch: 10/30... Step: 1270... Loss: 1.4303... Val Loss: 1.4313
Epoch: 10/30... Step: 1280... Loss: 1.4374... Val Loss: 1.4295
Epoch: 10/30... Step: 1290... Loss: 1.4295... Val Loss: 1.4280
Epoch: 10/30... Step: 1300... Loss: 1.4190... Val Loss: 1.4268
Epoch: 10/30... Step: 1310... Loss: 1.4223... Val Loss: 1.4222
Epoch: 10/30... Step: 1320... Loss: 1.3968... Val Loss: 1.4226
Epoch: 10/30... Step: 1330... Loss: 1.4015... Val Loss: 1.4207
Epoch: 10/30... Step: 1340... Loss: 1.3837... Val Loss: 1.4178
Epoch: 10/30... Step: 1350... Loss: 1.3768... Val Loss: 1.4181
Epoch: 10/30... Step: 1360... Loss: 1.3820... Val Loss: 1.4171
Epoch: 10/30... Step: 1370... Loss: 1.3722... Val Loss: 1.4128
Epoch: 10/30... Step: 1380... Loss: 1.4208... Val Loss: 1.4053
Epoch: 10/30... Step: 1390... Loss: 1.4207... Val Loss: 1.4080
Epoch: 11/30... Step: 1400... Loss: 1.4276... Val Loss: 1.4066
Epoch: 11/30... Step: 1410... Loss: 1.4383... Val Loss: 1.4041
Epoch: 11/30... Step: 1420... Loss: 1.4202... Val Loss: 1.4040
Epoch: 11/30... Step: 1430... Loss: 1.3815... Val Loss: 1.4043
Epoch: 11/30... Step: 1440... Loss: 1.4236... Val Loss: 1.4025
Epoch: 11/30... Step: 1450... Loss: 1.3349... Val Loss: 1.3978
Epoch: 11/30... Step: 1460... Loss: 1.3685... Val Loss: 1.3983
Epoch: 11/30... Step: 1470... Loss: 1.3618... Val Loss: 1.3972
Epoch: 11/30... Step: 1480... Loss: 1.3807... Val Loss: 1.3914
Epoch: 11/30... Step: 1490... Loss: 1.3628... Val Loss: 1.3948
Epoch: 11/30... Step: 1500... Loss: 1.3607... Val Loss: 1.3941
Epoch: 11/30... Step: 1510... Loss: 1.3341... Val Loss: 1.3939
Epoch: 11/30... Step: 1520... Loss: 1.3832... Val Loss: 1.3853
Epoch: 12/30... Step: 1530... Loss: 1.4313... Val Loss: 1.3864
Epoch: 12/30... Step: 1540... Loss: 1.3772... Val Loss: 1.3845
Epoch: 12/30... Step: 1550... Loss: 1.3865... Val Loss: 1.3825
Epoch: 12/30... Step: 1560... Loss: 1.3890... Val Loss: 1.3827
Epoch: 12/30... Step: 1570... Loss: 1.3446... Val Loss: 1.3831
Epoch: 12/30... Step: 1580... Loss: 1.3221... Val Loss: 1.3808
Epoch: 12/30... Step: 1590... Loss: 1.3236... Val Loss: 1.3770
Epoch: 12/30... Step: 1600... Loss: 1.3472... Val Loss: 1.3783
Epoch: 12/30... Step: 1610... Loss: 1.3426... Val Loss: 1.3806
Epoch: 12/30... Step: 1620... Loss: 1.3355... Val Loss: 1.3766
Epoch: 12/30... Step: 1630... Loss: 1.3553... Val Loss: 1.3731
Epoch: 12/30... Step: 1640... Loss: 1.3307... Val Loss: 1.3744
Epoch: 12/30... Step: 1650... Loss: 1.3111... Val Loss: 1.3740
Epoch: 12/30... Step: 1660... Loss: 1.3662... Val Loss: 1.3669
Epoch: 13/30... Step: 1670... Loss: 1.3306... Val Loss: 1.3697
Epoch: 13/30... Step: 1680... Loss: 1.3513... Val Loss: 1.3656
Epoch: 13/30... Step: 1690... Loss: 1.3278... Val Loss: 1.3639
Epoch: 13/30... Step: 1700... Loss: 1.3227... Val Loss: 1.3669
Epoch: 13/30... Step: 1710... Loss: 1.3162... Val Loss: 1.3659
Epoch: 13/30... Step: 1720... Loss: 1.3190... Val Loss: 1.3657
Epoch: 13/30... Step: 1730... Loss: 1.3597... Val Loss: 1.3603
Epoch: 13/30... Step: 1740... Loss: 1.3224... Val Loss: 1.3596
Epoch: 13/30... Step: 1750... Loss: 1.2883... Val Loss: 1.3643
Epoch: 13/30... Step: 1760... Loss: 1.3176... Val Loss: 1.3599
Epoch: 13/30... Step: 1770... Loss: 1.3435... Val Loss: 1.3570
Epoch: 13/30... Step: 1780... Loss: 1.3170... Val Loss: 1.3596
Epoch: 13/30... Step: 1790... Loss: 1.3066... Val Loss: 1.3526
Epoch: 13/30... Step: 1800... Loss: 1.3277... Val Loss: 1.3512
Epoch: 14/30... Step: 1810... Loss: 1.3285... Val Loss: 1.3523
Epoch: 14/30... Step: 1820... Loss: 1.3202... Val Loss: 1.3471
Epoch: 14/30... Step: 1830... Loss: 1.3275... Val Loss: 1.3466
Epoch: 14/30... Step: 1840... Loss: 1.2832... Val Loss: 1.3500
Epoch: 14/30... Step: 1850... Loss: 1.2642... Val Loss: 1.3499
Epoch: 14/30... Step: 1860... Loss: 1.3250... Val Loss: 1.3512
Epoch: 14/30... Step: 1870... Loss: 1.3345... Val Loss: 1.3483
Epoch: 14/30... Step: 1880... Loss: 1.3243... Val Loss: 1.3518
Epoch: 14/30... Step: 1890... Loss: 1.3428... Val Loss: 1.3515
Epoch: 14/30... Step: 1900... Loss: 1.3141... Val Loss: 1.3485
Epoch: 14/30... Step: 1910... Loss: 1.3127... Val Loss: 1.3427
Epoch: 14/30... Step: 1920... Loss: 1.3037... Val Loss: 1.3414
Epoch: 14/30... Step: 1930... Loss: 1.2703... Val Loss: 1.3394
Epoch: 14/30... Step: 1940... Loss: 1.3300... Val Loss: 1.3387
Epoch: 15/30... Step: 1950... Loss: 1.2971... Val Loss: 1.3405
Epoch: 15/30... Step: 1960... Loss: 1.3032... Val Loss: 1.3370
Epoch: 15/30... Step: 1970... Loss: 1.2914... Val Loss: 1.3331
Epoch: 15/30... Step: 1980... Loss: 1.2847... Val Loss: 1.3380
Epoch: 15/30... Step: 1990... Loss: 1.2822... Val Loss: 1.3344
Epoch: 15/30... Step: 2000... Loss: 1.2716... Val Loss: 1.3348
Epoch: 15/30... Step: 2010... Loss: 1.2980... Val Loss: 1.3343
Epoch: 15/30... Step: 2020... Loss: 1.3082... Val Loss: 1.3375
Epoch: 15/30... Step: 2030... Loss: 1.2770... Val Loss: 1.3394
Epoch: 15/30... Step: 2040... Loss: 1.2941... Val Loss: 1.3352
Epoch: 15/30... Step: 2050... Loss: 1.2763... Val Loss: 1.3284
Epoch: 15/30... Step: 2060... Loss: 1.2844... Val Loss: 1.3275
Epoch: 15/30... Step: 2070... Loss: 1.2927... Val Loss: 1.3231
Epoch: 15/30... Step: 2080... Loss: 1.2820... Val Loss: 1.3232
Epoch: 16/30... Step: 2090... Loss: 1.3042... Val Loss: 1.3281
Epoch: 16/30... Step: 2100... Loss: 1.2718... Val Loss: 1.3228
Epoch: 16/30... Step: 2110... Loss: 1.2687... Val Loss: 1.3221
Epoch: 16/30... Step: 2120... Loss: 1.2870... Val Loss: 1.3246
Epoch: 16/30... Step: 2130... Loss: 1.2618... Val Loss: 1.3242
Epoch: 16/30... Step: 2140... Loss: 1.2688... Val Loss: 1.3212
Epoch: 16/30... Step: 2150... Loss: 1.2985... Val Loss: 1.3216
Epoch: 16/30... Step: 2160... Loss: 1.2687... Val Loss: 1.3258
Epoch: 16/30... Step: 2170... Loss: 1.2644... Val Loss: 1.3220
Epoch: 16/30... Step: 2180... Loss: 1.2696... Val Loss: 1.3244
Epoch: 16/30... Step: 2190... Loss: 1.2799... Val Loss: 1.3198
Epoch: 16/30... Step: 2200... Loss: 1.2550... Val Loss: 1.3173
Epoch: 16/30... Step: 2210... Loss: 1.2377... Val Loss: 1.3159
Epoch: 16/30... Step: 2220... Loss: 1.2764... Val Loss: 1.3197
Epoch: 17/30... Step: 2230... Loss: 1.2468... Val Loss: 1.3161
Epoch: 17/30... Step: 2240... Loss: 1.2571... Val Loss: 1.3128
Epoch: 17/30... Step: 2250... Loss: 1.2438... Val Loss: 1.3126
Epoch: 17/30... Step: 2260... Loss: 1.2587... Val Loss: 1.3153
Epoch: 17/30... Step: 2270... Loss: 1.2640... Val Loss: 1.3119
Epoch: 17/30... Step: 2280... Loss: 1.2712... Val Loss: 1.3099
Epoch: 17/30... Step: 2290... Loss: 1.2785... Val Loss: 1.3133
Epoch: 17/30... Step: 2300... Loss: 1.2392... Val Loss: 1.3142
Epoch: 17/30... Step: 2310... Loss: 1.2492... Val Loss: 1.3150
Epoch: 17/30... Step: 2320... Loss: 1.2521... Val Loss: 1.3136
Epoch: 17/30... Step: 2330... Loss: 1.2441... Val Loss: 1.3100
Epoch: 17/30... Step: 2340... Loss: 1.2596... Val Loss: 1.3081
Epoch: 17/30... Step: 2350... Loss: 1.2650... Val Loss: 1.3022
Epoch: 17/30... Step: 2360... Loss: 1.2650... Val Loss: 1.3062
Epoch: 18/30... Step: 2370... Loss: 1.2375... Val Loss: 1.3050
Epoch: 18/30... Step: 2380... Loss: 1.2571... Val Loss: 1.3086
Epoch: 18/30... Step: 2390... Loss: 1.2496... Val Loss: 1.3056
Epoch: 18/30... Step: 2400... Loss: 1.2712... Val Loss: 1.3107
Epoch: 18/30... Step: 2410... Loss: 1.2717... Val Loss: 1.3060
Epoch: 18/30... Step: 2420... Loss: 1.2376... Val Loss: 1.3014
Epoch: 18/30... Step: 2430... Loss: 1.2507... Val Loss: 1.3027
Epoch: 18/30... Step: 2440... Loss: 1.2453... Val Loss: 1.3037
Epoch: 18/30... Step: 2450... Loss: 1.2297... Val Loss: 1.3013
Epoch: 18/30... Step: 2460... Loss: 1.2490... Val Loss: 1.3054
Epoch: 18/30... Step: 2470... Loss: 1.2453... Val Loss: 1.3013
Epoch: 18/30... Step: 2480... Loss: 1.2376... Val Loss: 1.3012
Epoch: 18/30... Step: 2490... Loss: 1.2321... Val Loss: 1.2965
Epoch: 18/30... Step: 2500... Loss: 1.2313... Val Loss: 1.2985
Epoch: 19/30... Step: 2510... Loss: 1.2377... Val Loss: 1.2966
Epoch: 19/30... Step: 2520... Loss: 1.2570... Val Loss: 1.3011
Epoch: 19/30... Step: 2530... Loss: 1.2470... Val Loss: 1.2976
Epoch: 19/30... Step: 2540... Loss: 1.2618... Val Loss: 1.2940
Epoch: 19/30... Step: 2550... Loss: 1.2314... Val Loss: 1.2979
Epoch: 19/30... Step: 2560... Loss: 1.2466... Val Loss: 1.2962
Epoch: 19/30... Step: 2570... Loss: 1.2261... Val Loss: 1.2968
Epoch: 19/30... Step: 2580... Loss: 1.2670... Val Loss: 1.2949
Epoch: 19/30... Step: 2590... Loss: 1.2205... Val Loss: 1.3005
Epoch: 19/30... Step: 2600... Loss: 1.2299... Val Loss: 1.2997
Epoch: 19/30... Step: 2610... Loss: 1.2311... Val Loss: 1.2967
Epoch: 19/30... Step: 2620... Loss: 1.2145... Val Loss: 1.2944
Epoch: 19/30... Step: 2630... Loss: 1.2189... Val Loss: 1.2894
Epoch: 19/30... Step: 2640... Loss: 1.2369... Val Loss: 1.2918
Epoch: 20/30... Step: 2650... Loss: 1.2357... Val Loss: 1.2909
Epoch: 20/30... Step: 2660... Loss: 1.2387... Val Loss: 1.2910
Epoch: 20/30... Step: 2670... Loss: 1.2443... Val Loss: 1.2895
Epoch: 20/30... Step: 2680... Loss: 1.2276... Val Loss: 1.2874
Epoch: 20/30... Step: 2690... Loss: 1.2314... Val Loss: 1.2907
Epoch: 20/30... Step: 2700... Loss: 1.2321... Val Loss: 1.2870
Epoch: 20/30... Step: 2710... Loss: 1.2090... Val Loss: 1.2925
Epoch: 20/30... Step: 2720... Loss: 1.2007... Val Loss: 1.2926
Epoch: 20/30... Step: 2730... Loss: 1.2041... Val Loss: 1.2912
Epoch: 20/30... Step: 2740... Loss: 1.2049... Val Loss: 1.2943
Epoch: 20/30... Step: 2750... Loss: 1.2016... Val Loss: 1.2891
Epoch: 20/30... Step: 2760... Loss: 1.2022... Val Loss: 1.2900
Epoch: 20/30... Step: 2770... Loss: 1.2391... Val Loss: 1.2874
Epoch: 20/30... Step: 2780... Loss: 1.2662... Val Loss: 1.2847
Epoch: 21/30... Step: 2790... Loss: 1.2395... Val Loss: 1.2863
Epoch: 21/30... Step: 2800... Loss: 1.2642... Val Loss: 1.2858
Epoch: 21/30... Step: 2810... Loss: 1.2477... Val Loss: 1.2814
Epoch: 21/30... Step: 2820... Loss: 1.2209... Val Loss: 1.2864
Epoch: 21/30... Step: 2830... Loss: 1.2393... Val Loss: 1.2910
Epoch: 21/30... Step: 2840... Loss: 1.1828... Val Loss: 1.2830
Epoch: 21/30... Step: 2850... Loss: 1.2092... Val Loss: 1.2829
Epoch: 21/30... Step: 2860... Loss: 1.1913... Val Loss: 1.2845
Epoch: 21/30... Step: 2870... Loss: 1.2151... Val Loss: 1.2825
Epoch: 21/30... Step: 2880... Loss: 1.2091... Val Loss: 1.2821
Epoch: 21/30... Step: 2890... Loss: 1.1936... Val Loss: 1.2817
Epoch: 21/30... Step: 2900... Loss: 1.1833... Val Loss: 1.2829
Epoch: 21/30... Step: 2910... Loss: 1.2161... Val Loss: 1.2790
Epoch: 22/30... Step: 2920... Loss: 1.2806... Val Loss: 1.2777
Epoch: 22/30... Step: 2930... Loss: 1.2210... Val Loss: 1.2860
Epoch: 22/30... Step: 2940... Loss: 1.2223... Val Loss: 1.2825
Epoch: 22/30... Step: 2950... Loss: 1.2408... Val Loss: 1.2747
Epoch: 22/30... Step: 2960... Loss: 1.1923... Val Loss: 1.2776
Epoch: 22/30... Step: 2970... Loss: 1.1813... Val Loss: 1.2817
Epoch: 22/30... Step: 2980... Loss: 1.1748... Val Loss: 1.2768
Epoch: 22/30... Step: 2990... Loss: 1.1888... Val Loss: 1.2819
Epoch: 22/30... Step: 3000... Loss: 1.1934... Val Loss: 1.2840
Epoch: 22/30... Step: 3010... Loss: 1.1813... Val Loss: 1.2808
Epoch: 22/30... Step: 3020... Loss: 1.2117... Val Loss: 1.2811
Epoch: 22/30... Step: 3030... Loss: 1.1839... Val Loss: 1.2769
Epoch: 22/30... Step: 3040... Loss: 1.1667... Val Loss: 1.2814
Epoch: 22/30... Step: 3050... Loss: 1.2244... Val Loss: 1.2740
Epoch: 23/30... Step: 3060... Loss: 1.1912... Val Loss: 1.2736
Epoch: 23/30... Step: 3070... Loss: 1.2029... Val Loss: 1.2776
Epoch: 23/30... Step: 3080... Loss: 1.1917... Val Loss: 1.2708
Epoch: 23/30... Step: 3090... Loss: 1.1780... Val Loss: 1.2694
Epoch: 23/30... Step: 3100... Loss: 1.1701... Val Loss: 1.2727
Epoch: 23/30... Step: 3110... Loss: 1.1861... Val Loss: 1.2784
Epoch: 23/30... Step: 3120... Loss: 1.2090... Val Loss: 1.2785
Epoch: 23/30... Step: 3130... Loss: 1.1814... Val Loss: 1.2753
Epoch: 23/30... Step: 3140... Loss: 1.1614... Val Loss: 1.2718
Epoch: 23/30... Step: 3150... Loss: 1.1802... Val Loss: 1.2736
Epoch: 23/30... Step: 3160... Loss: 1.2036... Val Loss: 1.2729
Epoch: 23/30... Step: 3170... Loss: 1.1739... Val Loss: 1.2708
Epoch: 23/30... Step: 3180... Loss: 1.1642... Val Loss: 1.2652
Epoch: 23/30... Step: 3190... Loss: 1.1958... Val Loss: 1.2650
Epoch: 24/30... Step: 3200... Loss: 1.1962... Val Loss: 1.2654
Epoch: 24/30... Step: 3210... Loss: 1.1749... Val Loss: 1.2648
Epoch: 24/30... Step: 3220... Loss: 1.2037... Val Loss: 1.2653
Epoch: 24/30... Step: 3230... Loss: 1.1508... Val Loss: 1.2580
Epoch: 24/30... Step: 3240... Loss: 1.1363... Val Loss: 1.2616
Epoch: 24/30... Step: 3250... Loss: 1.1887... Val Loss: 1.2633
Epoch: 24/30... Step: 3260... Loss: 1.1970... Val Loss: 1.2634
Epoch: 24/30... Step: 3270... Loss: 1.1911... Val Loss: 1.2634
Epoch: 24/30... Step: 3280... Loss: 1.2029... Val Loss: 1.2658
Epoch: 24/30... Step: 3290... Loss: 1.1776... Val Loss: 1.2596
Epoch: 24/30... Step: 3300... Loss: 1.1734... Val Loss: 1.2572
Epoch: 24/30... Step: 3310... Loss: 1.1784... Val Loss: 1.2613
Epoch: 24/30... Step: 3320... Loss: 1.1449... Val Loss: 1.2605
Epoch: 24/30... Step: 3330... Loss: 1.1909... Val Loss: 1.2518
Epoch: 25/30... Step: 3340... Loss: 1.1676... Val Loss: 1.2592
Epoch: 25/30... Step: 3350... Loss: 1.1747... Val Loss: 1.2558
Epoch: 25/30... Step: 3360... Loss: 1.1651... Val Loss: 1.2574
Epoch: 25/30... Step: 3370... Loss: 1.1589... Val Loss: 1.2499
Epoch: 25/30... Step: 3380... Loss: 1.1519... Val Loss: 1.2559
Epoch: 25/30... Step: 3390... Loss: 1.1454... Val Loss: 1.2588
Epoch: 25/30... Step: 3400... Loss: 1.1680... Val Loss: 1.2605
Epoch: 25/30... Step: 3410... Loss: 1.1746... Val Loss: 1.2596
Epoch: 25/30... Step: 3420... Loss: 1.1566... Val Loss: 1.2616
Epoch: 25/30... Step: 3430... Loss: 1.1776... Val Loss: 1.2558
Epoch: 25/30... Step: 3440... Loss: 1.1509... Val Loss: 1.2513
Epoch: 25/30... Step: 3450... Loss: 1.1514... Val Loss: 1.2558
Epoch: 25/30... Step: 3460... Loss: 1.1731... Val Loss: 1.2488
Epoch: 25/30... Step: 3470... Loss: 1.1701... Val Loss: 1.2469
Epoch: 26/30... Step: 3480... Loss: 1.1776... Val Loss: 1.2542
Epoch: 26/30... Step: 3490... Loss: 1.1511... Val Loss: 1.2534
Epoch: 26/30... Step: 3500... Loss: 1.1465... Val Loss: 1.2514
Epoch: 26/30... Step: 3510... Loss: 1.1658... Val Loss: 1.2519
Epoch: 26/30... Step: 3520... Loss: 1.1434... Val Loss: 1.2538
Epoch: 26/30... Step: 3530... Loss: 1.1572... Val Loss: 1.2571
Epoch: 26/30... Step: 3540... Loss: 1.1735... Val Loss: 1.2558
Epoch: 26/30... Step: 3550... Loss: 1.1578... Val Loss: 1.2570
Epoch: 26/30... Step: 3560... Loss: 1.1479... Val Loss: 1.2564
Epoch: 26/30... Step: 3570... Loss: 1.1570... Val Loss: 1.2556
Epoch: 26/30... Step: 3580... Loss: 1.1705... Val Loss: 1.2542
Epoch: 26/30... Step: 3590... Loss: 1.1427... Val Loss: 1.2480
Epoch: 26/30... Step: 3600... Loss: 1.1158... Val Loss: 1.2508
Epoch: 26/30... Step: 3610... Loss: 1.1563... Val Loss: 1.2482
Epoch: 27/30... Step: 3620... Loss: 1.1416... Val Loss: 1.2541
Epoch: 27/30... Step: 3630... Loss: 1.1395... Val Loss: 1.2494
Epoch: 27/30... Step: 3640... Loss: 1.1403... Val Loss: 1.2473
Epoch: 27/30... Step: 3650... Loss: 1.1499... Val Loss: 1.2522
Epoch: 27/30... Step: 3660... Loss: 1.1509... Val Loss: 1.2568
Epoch: 27/30... Step: 3670... Loss: 1.1607... Val Loss: 1.2588
Epoch: 27/30... Step: 3680... Loss: 1.1645... Val Loss: 1.2472
Epoch: 27/30... Step: 3690... Loss: 1.1259... Val Loss: 1.2523
Epoch: 27/30... Step: 3700... Loss: 1.1427... Val Loss: 1.2567
Epoch: 27/30... Step: 3710... Loss: 1.1358... Val Loss: 1.2538
Epoch: 27/30... Step: 3720... Loss: 1.1418... Val Loss: 1.2486
Epoch: 27/30... Step: 3730... Loss: 1.1521... Val Loss: 1.2502
Epoch: 27/30... Step: 3740... Loss: 1.1444... Val Loss: 1.2447
Epoch: 27/30... Step: 3750... Loss: 1.1554... Val Loss: 1.2432
Epoch: 28/30... Step: 3760... Loss: 1.1364... Val Loss: 1.2483
Epoch: 28/30... Step: 3770... Loss: 1.1336... Val Loss: 1.2435
Epoch: 28/30... Step: 3780... Loss: 1.1343... Val Loss: 1.2442
Epoch: 28/30... Step: 3790... Loss: 1.1610... Val Loss: 1.2466
Epoch: 28/30... Step: 3800... Loss: 1.1560... Val Loss: 1.2512
Epoch: 28/30... Step: 3810... Loss: 1.1338... Val Loss: 1.2515
Epoch: 28/30... Step: 3820... Loss: 1.1465... Val Loss: 1.2454
Epoch: 28/30... Step: 3830... Loss: 1.1357... Val Loss: 1.2536
Epoch: 28/30... Step: 3840... Loss: 1.1216... Val Loss: 1.2538
Epoch: 28/30... Step: 3850... Loss: 1.1391... Val Loss: 1.2511
Epoch: 28/30... Step: 3860... Loss: 1.1346... Val Loss: 1.2470
Epoch: 28/30... Step: 3870... Loss: 1.1249... Val Loss: 1.2480
Epoch: 28/30... Step: 3880... Loss: 1.1163... Val Loss: 1.2398
Epoch: 28/30... Step: 3890... Loss: 1.1299... Val Loss: 1.2450
Epoch: 29/30... Step: 3900... Loss: 1.1396... Val Loss: 1.2439
Epoch: 29/30... Step: 3910... Loss: 1.1469... Val Loss: 1.2392
Epoch: 29/30... Step: 3920... Loss: 1.1560... Val Loss: 1.2442
Epoch: 29/30... Step: 3930... Loss: 1.1557... Val Loss: 1.2393
Epoch: 29/30... Step: 3940... Loss: 1.1279... Val Loss: 1.2416
Epoch: 29/30... Step: 3950... Loss: 1.1278... Val Loss: 1.2428
Epoch: 29/30... Step: 3960... Loss: 1.1209... Val Loss: 1.2414
Epoch: 29/30... Step: 3970... Loss: 1.1627... Val Loss: 1.2417
Epoch: 29/30... Step: 3980... Loss: 1.1189... Val Loss: 1.2447
Epoch: 29/30... Step: 3990... Loss: 1.1226... Val Loss: 1.2420
Epoch: 29/30... Step: 4000... Loss: 1.1244... Val Loss: 1.2380
Epoch: 29/30... Step: 4010... Loss: 1.1116... Val Loss: 1.2436
Epoch: 29/30... Step: 4020... Loss: 1.1148... Val Loss: 1.2405
Epoch: 29/30... Step: 4030... Loss: 1.1365... Val Loss: 1.2422
Epoch: 30/30... Step: 4040... Loss: 1.1295... Val Loss: 1.2437
Epoch: 30/30... Step: 4050... Loss: 1.1462... Val Loss: 1.2380
Epoch: 30/30... Step: 4060... Loss: 1.1434... Val Loss: 1.2387
Epoch: 30/30... Step: 4070... Loss: 1.1251... Val Loss: 1.2432
Epoch: 30/30... Step: 4080... Loss: 1.1315... Val Loss: 1.2414
Epoch: 30/30... Step: 4090... Loss: 1.1296... Val Loss: 1.2392
Epoch: 30/30... Step: 4100... Loss: 1.1089... Val Loss: 1.2437
Epoch: 30/30... Step: 4110... Loss: 1.1082... Val Loss: 1.2437
Epoch: 30/30... Step: 4120... Loss: 1.1087... Val Loss: 1.2464
Epoch: 30/30... Step: 4130... Loss: 1.1034... Val Loss: 1.2467
Epoch: 30/30... Step: 4140... Loss: 1.1205... Val Loss: 1.2367
Epoch: 30/30... Step: 4150... Loss: 1.1037... Val Loss: 1.2412
Epoch: 30/30... Step: 4160... Loss: 1.1325... Val Loss: 1.2406
Epoch: 30/30... Step: 4170... Loss: 1.1650... Val Loss: 1.2374

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [ ]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [ ]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [ ]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [ ]:
print(sample(net, 1000, prime='Anna', top_k=5))

## Loading a checkpoint

In [ ]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [ ]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))